#subject_split_generator.ipynb
Generates a dictionary of train/test/validate subject numbers that can be passed to the load_data functions for assignment.

Author:  [Lee B. Hinkle](https://userweb.cs.txstate.edu/~lbh31/), IMICS Lab, Texas State University, 2021

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

TODO:
* Need to integrate code to handle cases like MobiAct that are non-contiguous
* Would be even better to import full subject list from X, y, **sub**



#Imports

In [ ]:
import shutil #for file manipulation
import random #to remove random sampling and create validation group
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import load
from pandas import read_csv
from matplotlib import pyplot
from tensorflow import keras #added to save model
from tensorflow.keras import layers #format matches MNIST example
from keras.utils import to_categorical

# to measure and display training time
import time
from datetime import timedelta

import matplotlib.pyplot as plt # for plotting - pandas uses matplotlib

#imports for computing and displaying output metrics
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

#Function:  split_sub returns subject dictionary containing nested dictionaries of train/validate/test subject numbers


In [ ]:
#this function pulls out num_subj_test subjects and will iterate through
#based on pass_num entered.
#useage:  if num_subj_test is 1 it will perform hold-one-subject out
#if more than one, it will hold-two-subjects-out etc.
def split_sub(
    all_sub = [2,4,6,8,10,12,14,16,18,20], 
    num_hold_out = 2,
    num_to_validation = 2,
    verbose = False,
    ):
    full_list = []
    test_sub_list = [all_sub[i * num_hold_out:(i + 1) * num_hold_out] for i in range((len(all_sub) + num_hold_out - 1) // num_hold_out )]
    for tsub_grp in test_sub_list:
        train_sub = [x for x in all_sub if x not in tsub_grp]
        validation_sub = random.choices(train_sub, k=num_to_validation)
        train_sub = [x for x in train_sub if x not in validation_sub]
        if (verbose):
            print("Result of new_split")
            print("     Train Subjects",train_sub)
            print("Validation Subjects",validation_sub)
            print(".     Test Subjects",tsub_grp)
        test_keys = ["train_subj", "validation_subj","test_subj"] 
        test_values = [train_sub,validation_sub,tsub_grp]
        new_split = dict(zip(test_keys, test_values))
        full_list.append(new_split)
    return full_list

In [ ]:
all_unimib_sub = list(range(1,31))
my_list = split_sub(all_sub = all_unimib_sub, num_to_validation = 0, num_hold_out = 1)
for i in my_list:
    print (i)

{'train_subj': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 'validation_subj': [], 'test_subj': [1]}
{'train_subj': [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 'validation_subj': [], 'test_subj': [2]}
{'train_subj': [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 'validation_subj': [], 'test_subj': [3]}
{'train_subj': [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 'validation_subj': [], 'test_subj': [4]}
{'train_subj': [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 'validation_subj': [], 'test_subj': [5]}
{'train_subj': [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 'validation_subj': [], 'test_subj': [6]}
{'tr